In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTEENN
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.impute import KNNImputer

In [10]:
df = pd.read_csv('ArbuzTrain.csv')
df.head()

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27,is_churn
0,301,49300,7,10.0,23.4286,0,9250.0,NaN,NaN,0,...,0.0,0.0,0.01,0.0,0.00,0.00,0.01,0.0,0.00,0
1,9394,58900,7,10.0,5.4286,0,24547.0,NaN,NaN,0,...,0.0,0.0,0.00,0.0,0.00,0.03,0.00,0.0,0.00,0
2,6680,43100,6,10.0,15.3333,0,6017.0,NaN,NaN,0,...,0.0,0.0,0.00,0.0,0.00,0.22,0.00,0.0,0.02,0
3,4774,33900,8,4.0,9.6250,0,7316.0,NaN,NaN,0,...,0.0,0.0,0.01,0.0,0.06,0.35,0.00,0.0,0.17,0
4,2722,37900,9,10.0,14.2222,0,2175.0,10.0,0.0,0,...,0.0,0.0,0.00,0.0,0.04,0.22,0.00,0.0,0.01,0


In [11]:
df.isnull().sum()

client_id                          0
average_check                      0
number_orders                      0
average_score                   1797
items_per_order                    0
is_friend                          0
last_check                         5
last_score                      7196
last_scored_after_order_days    7196
last_late_by_minutes               0
is_last_order_fullfilled           0
items_share_category_1             0
items_share_category_2             0
items_share_category_3             0
items_share_category_4             0
items_share_category_5             0
items_share_category_6             0
items_share_category_7             0
items_share_category_8             0
items_share_category_9             0
items_share_category_10            0
items_share_category_11            0
items_share_category_12            0
items_share_category_13            0
items_share_category_14            0
items_share_category_15            0
items_share_category_16            0
i

In [12]:
df=df.drop(['client_id', 'last_score', 'last_scored_after_order_days'],axis=1)

imputer = KNNImputer(n_neighbors = 3)
imputed_columns = imputer.fit_transform(df[['average_score', 'last_check']])

df[['average_score', 'last_check']] = imputed_columns

In [13]:
x=df.drop('is_churn',axis=1)
y=df['is_churn']

In [14]:
df['is_churn'].value_counts()

0    8505
1    2479
Name: is_churn, dtype: int64

**SMOTEENN**


In [15]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)

xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2, random_state = 111)

In [19]:
#RF
model_rf_smote=RandomForestClassifier(max_features='auto', n_estimators= 700, max_depth=50, min_samples_leaf = 1, min_samples_split = 2, bootstrap=False, class_weight = 'balanced')
model_rf_smote.fit(xr_train1,yr_train1)
yr_predict1 = model_rf_smote.predict(xr_test1)
print(classification_report(yr_test1, yr_predict1))
print('F1-score: ', f1_score(yr_test1, yr_predict1))
print(confusion_matrix(yr_test1, yr_predict1))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87       588
           1       0.90      0.90      0.90       757

    accuracy                           0.89      1345
   macro avg       0.89      0.89      0.89      1345
weighted avg       0.89      0.89      0.89      1345

F1-score:  0.9021164021164021
[[515  73]
 [ 75 682]]


In [20]:
#CSV OUTPUT
test = pd.read_csv('ArbuzTest.csv')

test=test.drop(['client_id', 'last_score', 'last_scored_after_order_days'],axis=1)
imputed_test_columns = imputer.fit_transform(test[['average_score', 'last_check']])
test[['average_score', 'last_check']] = imputed_test_columns

In [21]:
yr_predict1 = model_rf_smote.predict(test)

In [22]:
test = pd.read_csv('ArbuzTest.csv')
test['is_churn'] = yr_predict1
final = test[['client_id', 'is_churn']]
final.to_csv('final_result.csv', index = False)